In [1]:
# lstm_lstm: emotion recognition from speech= lstm, text=lstm
# created for ATSIT paper 2020
# coded by Bagus Tris Atmaja (bagus@ep.its.ac.id)

import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import random as rn
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

Using TensorFlow backend.
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/root/anaconda3/envs/ser36/lib/python3.6/sit

In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:

# load feature and labels
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')
# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

# standardization
scaled_feature = False

# text feature
#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
data = np.load('../../dimensionalSM/IEMOCAP_full_release/data_collected_10039.pickle', allow_pickle=True)
text = [t['transcription'] for t in data]
#vad = np.load('../../data/y_egemaps.npy')
# load fastext model
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_train_text = []
x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

word_index = tokenizer.word_index
nb_words = len(word_index) +1
print('Found %s unique tokens' % len(word_index))
#g_word_embedding_matrix = np.load(path+'g_word_embedding_matrix.npy')

# other parameters
# MAX_SEQUENCE_LENGTH = 554
EMBEDDING_DIM = 300
# nb_words = 3438



Found 3437 unique tokens


In [4]:
if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1, keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss

In [5]:
# split train/test
split = 7869

In [6]:
# API model
def model():
    # speech network
    input_speech = Input(shape=(feat.shape[1], feat.shape[2]), name='speech_input')
    net_speech = BatchNormalization()(input_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(128, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    model_speech = Flatten()(net_speech)
    #model_speech = Dropout(0.1)(net_speech)
    
    #text network
    input_text = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net_text = Embedding(nb_words,
                    EMBEDDING_DIM,
                    trainable = True)(input_text)
    net = Bidirectional(LSTM(128, return_sequences=True))(net_text)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    model_text = Dense(64)(net)

    # combined model
    model_combined = concatenate([model_speech, model_text])
    model_combined = Dense(64, activation='relu')(model_combined)
    model_combined = Dense(32, activation='relu')(model_combined)
    #model_combined = Dropout(0.4)(model_combined)
    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_combined) for name in target_names]

    model = Model([input_speech, input_text], model_combined) 
    model.compile(loss=ccc_loss,
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


#model_1.summary()

In [7]:
# main function for for LOSO
def main():
    model_1 = model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [feat[:split],x_train_text[:split]],
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [feat[split:],x_train_text[split:]], vad[split:].T.tolist())
    print(metrik[-3:])
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(30):
        best = main()
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('speech_textWE_multi7869.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 57s 9ms/step - loss: 1.5374 - v_loss: 0.6047 - a_loss: 0.4240 - d_loss: 0.5082 - v_ccc: 0.3949 - a_ccc: 0.5759 - d_ccc: 0.4918 - val_loss: 2.0052 - val_v_loss: 0.6619 - val_a_loss: 0.6893 - val_d_loss: 0.6619 - val_v_ccc: 0.3458 - val_a_ccc: 0.3138 - val_d_ccc: 0.3353
Epoch 2/50
6295/6295 [==============================] - 50s 8ms/step - loss: 1.0870 - v_loss: 0.3600 - a_loss: 0.3204 - d_loss: 0.4068 - v_ccc: 0.6402 - a_ccc: 0.6795 - d_ccc: 0.5933 - val_loss: 1.6615 - val_v_loss: 0.6149 - val_a_loss: 0.5203 - val_d_loss: 0.5332 - val_v_ccc: 0.3897 - val_a_ccc: 0.4824 - val_d_ccc: 0.4664
Epoch 3/50
6295/6295 [==============================] - 50s 8ms/step - loss: 0.9627 - v_loss: 0.2951 - a_loss: 0.2932 - d_loss: 0.3742 - v_ccc: 0.7049 - a_ccc: 0.7066 - d_ccc: 0.6258

Epoch 9/50
6295/6295 [==============================] - 49s 8ms/step - loss: 0.6257 - v_loss: 0.1707 - a_loss: 0.2135 - d_loss: 0.2417 - v_ccc: 0.8293 - a_ccc: 0.7864 - d_ccc: 0.7586 - val_loss: 1.5692 - val_v_loss: 0.5497 - val_a_loss: 0.4675 - val_d_loss: 0.5674 - val_v_ccc: 0.4592 - val_a_ccc: 0.5374 - val_d_ccc: 0.4342
Epoch 10/50
6295/6295 [==============================] - 49s 8ms/step - loss: 0.5914 - v_loss: 0.1597 - a_loss: 0.1999 - d_loss: 0.2318 - v_ccc: 0.8403 - a_ccc: 0.8002 - d_ccc: 0.7681 - val_loss: 1.5962 - val_v_loss: 0.5520 - val_a_loss: 0.4840 - val_d_loss: 0.5763 - val_v_ccc: 0.4589 - val_a_ccc: 0.5185 - val_d_ccc: 0.4265
Epoch 11/50
6295/6295 [==============================] - 49s 8ms/step - loss: 0.5499 - v_loss: 0.1544 - a_loss: 0.1835 - d_loss: 0.2117 - v_ccc: 0.8455 - a_ccc: 0.8165 - d_ccc: 0.7882 - val_loss: 1.5836 - val_v_loss: 0.5532 - val_a_loss: 0.4713 - val_d_loss: 0.5753 - val_v_ccc: 0.4557 - val_a_ccc: 0.5335 - val_d_ccc: 0.4272
Epoch 12/50
6295/6295 [

6295/6295 [==============================] - 49s 8ms/step - loss: 1.0977 - v_loss: 0.3612 - a_loss: 0.3252 - d_loss: 0.4114 - v_ccc: 0.6389 - a_ccc: 0.6750 - d_ccc: 0.5884 - val_loss: 1.6566 - val_v_loss: 0.6585 - val_a_loss: 0.4985 - val_d_loss: 0.5106 - val_v_ccc: 0.3485 - val_a_ccc: 0.5034 - val_d_ccc: 0.4916
Epoch 3/50
6295/6295 [==============================] - 49s 8ms/step - loss: 0.9741 - v_loss: 0.3038 - a_loss: 0.3004 - d_loss: 0.3706 - v_ccc: 0.6960 - a_ccc: 0.6999 - d_ccc: 0.6300 - val_loss: 1.5686 - val_v_loss: 0.6191 - val_a_loss: 0.4361 - val_d_loss: 0.5296 - val_v_ccc: 0.3897 - val_a_ccc: 0.5659 - val_d_ccc: 0.4759
Epoch 4/50
6295/6295 [==============================] - 49s 8ms/step - loss: 0.8916 - v_loss: 0.2644 - a_loss: 0.2824 - d_loss: 0.3446 - v_ccc: 0.7355 - a_ccc: 0.7176 - d_ccc: 0.6553 - val_loss: 1.5171 - val_v_loss: 0.5989 - val_a_loss: 0.4217 - val_d_loss: 0.5124 - val_v_ccc: 0.4098 - val_a_ccc: 0.5825 - val_d_ccc: 0.4905
Epoch 5/50
6295/6295 [==============

/root/anaconda3/envs/ser36/lib/python3.6/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.211782). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


6295/6295 [==============================] - 48s 8ms/step - loss: 0.5602 - v_loss: 0.1527 - a_loss: 0.1895 - d_loss: 0.2183 - v_ccc: 0.8474 - a_ccc: 0.8106 - d_ccc: 0.7818 - val_loss: 1.5763 - val_v_loss: 0.5592 - val_a_loss: 0.4554 - val_d_loss: 0.5787 - val_v_ccc: 0.4516 - val_a_ccc: 0.5489 - val_d_ccc: 0.4232
Epoch 12/50
6295/6295 [==============================] - 48s 8ms/step - loss: 0.5337 - v_loss: 0.1468 - a_loss: 0.1815 - d_loss: 0.2053 - v_ccc: 0.8532 - a_ccc: 0.8184 - d_ccc: 0.7947 - val_loss: 1.5435 - val_v_loss: 0.5449 - val_a_loss: 0.4594 - val_d_loss: 0.5546 - val_v_ccc: 0.4650 - val_a_ccc: 0.5444 - val_d_ccc: 0.4472
Epoch 13/50
2170/2170 [==============================] - 5s 2ms/step
[0.3927529752254486, 0.5714371204376221, 0.5048881769180298]
4
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 52s 8ms/step - loss: 1.5404 - v_loss: 0.6032 - a_loss: 0.4149 - d_loss: 0.5218 - v_ccc: 0.3965 - a_ccc: 0.5849 - d_ccc: 0.47

...

6295/6295 [==============================] - 47s 8ms/step - loss: 0.5546 - v_loss: 0.1521 - a_loss: 0.1856 - d_loss: 0.2169 - v_ccc: 0.8480 - a_ccc: 0.8144 - d_ccc: 0.7829 - val_loss: 1.6186 - val_v_loss: 0.5621 - val_a_loss: 0.4806 - val_d_loss: 0.5910 - val_v_ccc: 0.4458 - val_a_ccc: 0.5204 - val_d_ccc: 0.4152
Epoch 12/50
6295/6295 [==============================] - 47s 8ms/step - loss: 0.5156 - v_loss: 0.1398 - a_loss: 0.1761 - d_loss: 0.1996 - v_ccc: 0.8601 - a_ccc: 0.8240 - d_ccc: 0.8003 - val_loss: 1.6023 - val_v_loss: 0.5696 - val_a_loss: 0.4679 - val_d_loss: 0.5872 - val_v_ccc: 0.4408 - val_a_ccc: 0.5365 - val_d_ccc: 0.4204
Epoch 13/50
2170/2170 [==============================] - 5s 2ms/step
[0.3686234951019287, 0.5708373188972473, 0.4857178330421448]
23
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 66s 10ms/step - loss: 1.5291 - v_loss: 0.5920 - a_loss: 0.4198 - d_loss: 0.5166 - v_ccc: 0.4074 - a_ccc: 0.5803 - d_ccc: 0.

6295/6295 [==============================] - 48s 8ms/step - loss: 0.8940 - v_loss: 0.2682 - a_loss: 0.2811 - d_loss: 0.3445 - v_ccc: 0.7316 - a_ccc: 0.7189 - d_ccc: 0.6554 - val_loss: 1.5254 - val_v_loss: 0.5862 - val_a_loss: 0.4110 - val_d_loss: 0.5386 - val_v_ccc: 0.4191 - val_a_ccc: 0.5900 - val_d_ccc: 0.4655
Epoch 5/50
6295/6295 [==============================] - 48s 8ms/step - loss: 0.8331 - v_loss: 0.2383 - a_loss: 0.2708 - d_loss: 0.3235 - v_ccc: 0.7615 - a_ccc: 0.7290 - d_ccc: 0.6764 - val_loss: 1.5361 - val_v_loss: 0.5684 - val_a_loss: 0.4347 - val_d_loss: 0.5471 - val_v_ccc: 0.4379 - val_a_ccc: 0.5674 - val_d_ccc: 0.4586
Epoch 6/50
6295/6295 [==============================] - 48s 8ms/step - loss: 0.7686 - v_loss: 0.2187 - a_loss: 0.2502 - d_loss: 0.3002 - v_ccc: 0.7813 - a_ccc: 0.7499 - d_ccc: 0.7001 - val_loss: 1.5926 - val_v_loss: 0.5890 - val_a_loss: 0.4503 - val_d_loss: 0.5666 - val_v_ccc: 0.4189 - val_a_ccc: 0.5495 - val_d_ccc: 0.4390
Epoch 7/50
6295/6295 [==============

Epoch 16/50
6295/6295 [==============================] - 47s 8ms/step - loss: 0.4398 - v_loss: 0.1211 - a_loss: 0.1523 - d_loss: 0.1662 - v_ccc: 0.8788 - a_ccc: 0.8477 - d_ccc: 0.8337 - val_loss: 1.6112 - val_v_loss: 0.5533 - val_a_loss: 0.4863 - val_d_loss: 0.5875 - val_v_ccc: 0.4570 - val_a_ccc: 0.5162 - val_d_ccc: 0.4155
Epoch 17/50
2170/2170 [==============================] - 5s 2ms/step
[0.41671520471572876, 0.5771441459655762, 0.4869615435600281]
26
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 68s 11ms/step - loss: 1.5363 - v_loss: 0.6091 - a_loss: 0.4197 - d_loss: 0.5076 - v_ccc: 0.3909 - a_ccc: 0.5805 - d_ccc: 0.4924 - val_loss: 2.0189 - val_v_loss: 0.6897 - val_a_loss: 0.7172 - val_d_loss: 0.6256 - val_v_ccc: 0.3163 - val_a_ccc: 0.2860 - val_d_ccc: 0.3788
Epoch 2/50
6295/6295 [==============================] - 51s 8ms/step - loss: 1.0922 - v_loss: 0.3618 - a_loss: 0.3242 - d_loss: 0.4062 - v_ccc: 0.6381 - a_ccc: 0.6758

6295/6295 [==============================] - 48s 8ms/step - loss: 0.6738 - v_loss: 0.1883 - a_loss: 0.2224 - d_loss: 0.2630 - v_ccc: 0.8117 - a_ccc: 0.7775 - d_ccc: 0.7370 - val_loss: 1.6020 - val_v_loss: 0.5815 - val_a_loss: 0.4661 - val_d_loss: 0.5776 - val_v_ccc: 0.4291 - val_a_ccc: 0.5388 - val_d_ccc: 0.4301
Epoch 9/50
6295/6295 [==============================] - 47s 8ms/step - loss: 0.6378 - v_loss: 0.1764 - a_loss: 0.2120 - d_loss: 0.2494 - v_ccc: 0.8237 - a_ccc: 0.7879 - d_ccc: 0.7506 - val_loss: 1.5840 - val_v_loss: 0.5706 - val_a_loss: 0.4563 - val_d_loss: 0.5746 - val_v_ccc: 0.4391 - val_a_ccc: 0.5482 - val_d_ccc: 0.4287
Epoch 10/50
6295/6295 [==============================] - 48s 8ms/step - loss: 0.5963 - v_loss: 0.1646 - a_loss: 0.1996 - d_loss: 0.2323 - v_ccc: 0.8354 - a_ccc: 0.8005 - d_ccc: 0.7679 - val_loss: 1.5313 - val_v_loss: 0.5565 - val_a_loss: 0.4538 - val_d_loss: 0.5396 - val_v_ccc: 0.4518 - val_a_ccc: 0.5512 - val_d_ccc: 0.4656
Epoch 11/50
6295/6295 [============

Epoch 13/50
6295/6295 [==============================] - 48s 8ms/step - loss: 0.5001 - v_loss: 0.1383 - a_loss: 0.1701 - d_loss: 0.1926 - v_ccc: 0.8621 - a_ccc: 0.8300 - d_ccc: 0.8078 - val_loss: 1.7212 - val_v_loss: 0.5734 - val_a_loss: 0.5257 - val_d_loss: 0.6352 - val_v_ccc: 0.4368 - val_a_ccc: 0.4786 - val_d_ccc: 0.3635
Epoch 14/50
6295/6295 [==============================] - 47s 8ms/step - loss: 0.4818 - v_loss: 0.1350 - a_loss: 0.1656 - d_loss: 0.1814 - v_ccc: 0.8650 - a_ccc: 0.8346 - d_ccc: 0.8185 - val_loss: 1.6472 - val_v_loss: 0.5680 - val_a_loss: 0.4728 - val_d_loss: 0.6180 - val_v_ccc: 0.4422 - val_a_ccc: 0.5282 - val_d_ccc: 0.3824
Epoch 15/50
6295/6295 [==============================] - 48s 8ms/step - loss: 0.4577 - v_loss: 0.1263 - a_loss: 0.1560 - d_loss: 0.1755 - v_ccc: 0.8738 - a_ccc: 0.8440 - d_ccc: 0.8245 - val_loss: 1.6071 - val_v_loss: 0.5710 - val_a_loss: 0.4756 - val_d_loss: 0.5769 - val_v_ccc: 0.4420 - val_a_ccc: 0.5225 - val_d_ccc: 0.4284
Epoch 16/50
2170/2170 